In [1]:
import pandas as pd
import numpy as np
import math


In [2]:
# Importando .csv
train = pd.read_csv('dataset/train_data.csv')

# Criando Matrix Movie X User
df_m_u = train.pivot(index='movie_id', columns='user_id', values='rating')

In [3]:
# Calculando correlação de pearson
sim_u_v = df_m_u.corr(method='pearson')

In [112]:
# Implementando predição
def predict(u, i, k = 5):
    sim_users = sim_u_v[u][~sim_u_v[u].isna()].sort_values(ascending=False).index
    rated_items = df_m_u.T[i][df_m_u.T[i] > 0].index
    sim_k = np.intersect1d(sim_users, rated_items)
    top_k = [x for x in sim_users if x in sim_k][:k]
    sum_sim = 0
    dem = 0
    mean_u = df_m_u[u].mean(skipna = True)
    for v in top_k:
        dem += sim_u_v[u][v] * (df_m_u[v][i] - df_m_u[v].mean(skipna = True))
        sum_sim += sim_u_v[u][v]
    if sum_sim == 0:
        return 0
    return mean_u + (dem/sum_sim)

In [113]:
# Avaliando modelo
def rmse(test):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(u, i)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [116]:
rmse(train.values[:1000])

0.5940892599225607